In [0]:
use catalog pricing_analytics;
use schema silver;

create or replace table silver.reporting_dim_variety_stage_1 AS 
select 
distinct variety from  silver.daily_pricing_silver
where lakehouse_updated_date > 
(select nvl(max(PROCESSED_TABLE_DATE_TIME), "1900-01-01") 
FROM pricing_analytics.processrunlogs.deltalakehouse_process_runs
WHERE process_name = 'reportingDimensionTablesLoad' AND process_status = 'Completed'
);


In [0]:
create or replace table pricing_analytics.silver.reporting_dim_variety_stage_2 AS 
select 
       silverDim.variety,
       (select nvl(max(variety_ID), 0 ) from pricing_analytics.gold.reporting_dim_variety_gold)+ row_number() over(order by silverDim.variety) as variety_ID,
       current_timestamp() as lakehouse_inserted_date,
       current_timestamp() as lakehouse_updated_date
       FROM pricing_analytics.silver.reporting_dim_variety_stage_1 silverDim  
       LEFT JOIN pricing_analytics.gold.reporting_dim_variety_gold goldDim
       on silverDim.variety = golddim.variety    
       where isnull(golddim.variety)
       ;

In [0]:
insert into pricing_analytics.gold.reporting_dim_variety_gold
select * from pricing_analytics.silver.reporting_dim_variety_stage_2;

In [0]:
select * from  pricing_analytics.gold.reporting_dim_variety_gold ; 